https://keras.io/examples/nlp/lstm_seq2seq/

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip

['Archive:  fra-eng.zip',
 'replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y',
 '  inflating: _about.txt              ',
 'replace fra.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y',
 '  inflating: fra.txt                 ']

In [ ]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000

data_path = "fra.txt"

In [ ]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
  lines = f.read().split('\n')
lines[:5]

['Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)',
 'Go.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)',
 'Go.\tBouge !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)',
 'Hi.\tSalut !\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)',
 'Hi.\tSalut.\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)']

In [ ]:
for line in lines[:min(num_samples,len(lines)-1)]:
  input_text, target_text,_ = line.split('\t')

  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encode_seq_length = max([len(txt) for txt in input_texts])
max_decode_seq_length = max([len(txt) for txt in target_texts])
print(input_characters)
print(target_characters)

[' ', '!', '"', '$', '%', '&', "'", ',', '-', '.', '0', '1', '2', '3', '5', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é']
['\t', '\n', ' ', '!', '%', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '5', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '«', '»', 'À', 'Ç', 'É', 'Ê', 'à', 'â', 'ç', 'è', 'é', 'ê', 'î', 'ï', 'ô', 'ù', 'û', 'œ', '\u2009', '’', '\u202f']


In [ ]:
print("Number of samples",len(input_texts))
print("Number of unique input tokens",num_encoder_tokens)
print("Number of unique target tokens",num_decoder_tokens)
print("Max Sequence length for inputs",max_encode_seq_length)
print("Max Sequence length for outputs",max_decode_seq_length)

Number of samples 10000
Number of unique input tokens 71
Number of unique target tokens 92
Max Sequence length for inputs 15
Max Sequence length for outputs 59


In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
print(input_token_index)
print(target_token_index)

{' ': 0, '!': 1, '"': 2, '$': 3, '%': 4, '&': 5, "'": 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '5': 14, '7': 15, '8': 16, '9': 17, ':': 18, '?': 19, 'A': 20, 'B': 21, 'C': 22, 'D': 23, 'E': 24, 'F': 25, 'G': 26, 'H': 27, 'I': 28, 'J': 29, 'K': 30, 'L': 31, 'M': 32, 'N': 33, 'O': 34, 'P': 35, 'Q': 36, 'R': 37, 'S': 38, 'T': 39, 'U': 40, 'V': 41, 'W': 42, 'Y': 43, 'a': 44, 'b': 45, 'c': 46, 'd': 47, 'e': 48, 'f': 49, 'g': 50, 'h': 51, 'i': 52, 'j': 53, 'k': 54, 'l': 55, 'm': 56, 'n': 57, 'o': 58, 'p': 59, 'q': 60, 'r': 61, 's': 62, 't': 63, 'u': 64, 'v': 65, 'w': 66, 'x': 67, 'y': 68, 'z': 69, 'é': 70}
{'\t': 0, '\n': 1, ' ': 2, '!': 3, '%': 4, '&': 5, "'": 6, '(': 7, ')': 8, ',': 9, '-': 10, '.': 11, '0': 12, '1': 13, '2': 14, '3': 15, '5': 16, '8': 17, '9': 18, ':': 19, '?': 20, 'A': 21, 'B': 22, 'C': 23, 'D': 24, 'E': 25, 'F': 26, 'G': 27, 'H': 28, 'I': 29, 'J': 30, 'K': 31, 'L': 32, 'M': 33, 'N': 34, 'O': 35, 'P': 36, 'Q': 37, 'R': 38, 'S': 39, 'T': 40, 'U': 41,

In [ ]:
encoder_input_data = np.zeros((len(input_texts),max_encode_seq_length,num_encoder_tokens),dtype="float32")
decoder_input_data = np.zeros((len(input_texts),max_decode_seq_length,num_decoder_tokens),dtype="float32")
decoder_target_data = np.zeros((len(input_texts),max_decode_seq_length,num_decoder_tokens),dtype="float32")

for i, (input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i,t,input_token_index[char]] = 1.0
  encoder_input_data[i,t+1 :,input_token_index[" "]] = 1.0
  for t, char in enumerate(target_text):
    decoder_input_data[i,t,target_token_index[char]] = 1.0
    if t>0:
      decoder_target_data[i,t-1,target_token_index[char]] = 1.0
  decoder_input_data[i,t+1 :,target_token_index[" "]] = 1.0
  decoder_target_data[i,t:,target_token_index[" "]] = 1.0


In [ ]:
encoder_input_data.shape

(10000, 15, 71)

#Model

In [ ]:
#Inut Sequence
encoder_inputs = keras.Input(shape=(None,num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

#Discard encoder outputs
encoder_states = [state_h,state_c]

#Decoder states
decoder_inputs = keras.Input(shape=(None,num_decoder_tokens))

#Set up decoder to return full output sequence
decoder_lstm = keras.layers.LSTM(latent_dim,return_sequences=True, return_state=True)
decoder_outputs, _,_ = decoder_lstm(decoder_inputs,initial_state=encoder_states)

decoder_dense = keras.layers.Dense(num_decoder_tokens,activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs,decoder_inputs],decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)
model.save("s2s")

Epoch 1/100
125/125 [==============================] - 5s 17ms/step - loss: 1.1351 - accuracy: 0.7366 - val_loss: 1.0440 - val_accuracy: 0.7144
Epoch 2/100
125/125 [==============================] - 1s 11ms/step - loss: 0.8141 - accuracy: 0.7786 - val_loss: 0.8346 - val_accuracy: 0.7703
Epoch 3/100
125/125 [==============================] - 1s 11ms/step - loss: 0.6691 - accuracy: 0.8138 - val_loss: 0.7140 - val_accuracy: 0.7977
Epoch 4/100
125/125 [==============================] - 1s 12ms/step - loss: 0.5833 - accuracy: 0.8311 - val_loss: 0.6378 - val_accuracy: 0.8149
Epoch 5/100
125/125 [==============================] - 1s 11ms/step - loss: 0.5370 - accuracy: 0.8426 - val_loss: 0.6045 - val_accuracy: 0.8238
Epoch 6/100
125/125 [==============================] - 1s 11ms/step - loss: 0.5003 - accuracy: 0.8532 - val_loss: 0.5665 - val_accuracy: 0.8354
Epoch 7/100
125/125 [==============================] - 1s 11ms/step - loss: 0.4705 - accuracy: 0.8611 - val_loss: 0.5437 - val_accuracy:

INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


#Inference

In [ ]:
model = keras.models.load_model("s2s")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 92)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 335872      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  357376      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [ ]:
model.layers

In [ ]:
model.input

[<KerasTensor: shape=(None, None, 71) dtype=float32 (created by layer 'input_1')>,
 <KerasTensor: shape=(None, None, 92) dtype=float32 (created by layer 'input_2')>]

In [ ]:
encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output 
encoder_states = [state_h_enc,state_c_enc]
encoder_model = keras.Model(encoder_inputs,encoder_states)

In [ ]:
decoder_inputs = model.input[1]


In [ ]:
model.layers[2].output

[<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm')>]

In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decode_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [ ]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Marche.

-
Input sentence: Go.
Decoded sentence: Marche.

-
Input sentence: Go.
Decoded sentence: Marche.

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Filez !

-
Input sentence: Run!
Decoded sentence: Filez !

-
Input sentence: Run!
Decoded sentence: Filez !

-
Input sentence: Run!
Decoded sentence: Filez !

-
Input sentence: Run!
Decoded sentence: Filez !

-
Input sentence: Run!
Decoded sentence: Filez !

-
Input sentence: Run!
Decoded sentence: Filez !

-
Input sentence: Run!
Decoded sentence: Filez !

-
Input sentence: Run.
Decoded sentence: Filez !

-
Input sentence: Run.
Decoded sentence: Filez !

-
Input sentence: Run.
Decoded sentence: Filez !

-
Input sentence: Run.
Decoded sentence: Filez !

-
Input sentence: Run.
Decoded sentence: Filez !

-
Input sentence: Run.
Decoded sentence: Filez !

-
Input sentence: Run.
Decoded sentence: Filez !



In [ ]:
!pip install transformers

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
model = AutoModelForCausalLM.from_pretrained('distilgpt2',output_hidden_states=True)

In [17]:
text = "I am fine but I can't walk"

input = tokenizer.encode(text,return_tensors="pt")
print(input)

output = model.generate(input,max_length=5,do_sample=False)
print(output)
print('\n',tokenizer.decode(output[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 8, but ``max_length`` is set to 5.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


tensor([[  40,  716, 3734,  475,  314,  460,  470, 2513]])
tensor([[  40,  716, 3734,  475,  314,  460,  470, 2513,  287]])

 I am fine but I can't walk in
